In [1]:
import pandas as pd

In [2]:
df = (pd
    .read_html("https://www.resultats-elections.interieur.gouv.fr/presidentielle-2022/FE.html",
        thousands=" ",
        decimal=",")
    [1]
    .assign(**{
            "diff": lambda df_: (df_["Voix"]
                .rolling(window=2)
                .apply(lambda s : -(s.iloc[0] - s.iloc[1])))
    })
)

df

,Liste des candidats,Voix,% Inscrits,% Exprimés,diff
0,M. Emmanuel MACRON,7939617,20.06,27.38,NaN
1,Mme Marine LE PEN,7390659,18.67,25.49,-548958.0
2,M. Jean-Luc MÉLENCHON,5781061,14.60,19.94,-1609598.0
3,M. Éric ZEMMOUR,2004536,5.06,6.91,-3776525.0
4,Mme Valérie PÉCRESSE,1368445,3.46,4.72,-636091.0
5,M. Yannick JADOT,1263509,3.19,4.36,-104936.0
6,M. Jean LASSALLE,1002658,2.53,3.46,-260851.0
7,M. Fabien ROUSSEL,690102,1.74,2.38,-312556.0
8,M. Nicolas DUPONT-AIGNAN,639328,1.61,2.21,-50774.0
9,Mme Anne HIDALGO,509353,1.29,1.76,-129975.0
